In [ ]:
import pandas as pd
import requests
import os
import json

In [ ]:
URL = "https://opendata.comune.bologna.it/api/explore/v2.1/catalog/datasets/disponibilita-parcheggi-storico/exports/csv?lang=it&timezone=UTC&use_labels=true&delimiter=%3B"

df = pd.read_csv(URL, sep=";")
df[['lat', 'lon']] = df['coordinate'].str.split(', ',expand=True)
df = df.drop(columns=['% occupazione', 'GUID', 'coordinate']).rename(columns={'Parcheggio': 'parcheggio', 'Data': 'data', 'Posti liberi': 'posti_liberi', 'Posti occupati': 'posti_occupati', 'Posti totali': 'posti_totali'})
df

In [ ]:
KEYS = ['parcheggio', 'lat', 'lon']
df_parcheggi = df.groupby(['parcheggio']).first().reset_index()[KEYS]
df_parcheggi

In [ ]:
rdf = df.copy()
rdf['data'] = pd.to_datetime(rdf['data'])
rdf['day'] = rdf['data'].apply(lambda t: t.replace(second=0, minute=0))
rdf['lat'] = rdf['lat'].apply(lambda t: float(t))
rdf['lon'] = rdf['lon'].apply(lambda t: float(t))
rdf = rdf.drop(columns=['data'])
grouped =rdf.groupby(['parcheggio','day']).mean()
df_aggregated = grouped.reset_index()
df_aggregated

In [ ]:
import digitalhub as dh

PROJECT_NAME = "parcheggi"
proj = dh.get_or_create_project(PROJECT_NAME) # source="git://github.com/scc-digitalhub/gdb-project-parkings.git"

In [ ]:
%%writefile "src/download-all.py"
from digitalhub_runtime_python import handler
import pandas as pd

@handler(outputs=["dataset"])
def downloader(project, url):
    df = url.as_df(file_format='csv',sep=";")
    df[['lat', 'lon']] = df['coordinate'].str.split(', ',expand=True)
    df = df.drop(columns=['% occupazione', 'GUID', 'coordinate']).rename(columns={'Parcheggio': 'parcheggio', 'Data': 'data', 'Posti liberi': 'posti_liberi', 'Posti occupati': 'posti_occupati', 'Posti totali': 'posti_totali'})
    df["lat"] = pd.to_numeric(df["lat"])
    df["lon"] = pd.to_numeric(df["lon"])
    return df

In [ ]:
FUNCTION_NAME="downloader-funct"
func = proj.new_function(name=FUNCTION_NAME,
                         kind="python",
                         python_version="PYTHON3_9",
                         source={"source": "src/download-all.py", "handler": "downloader"})

In [ ]:
di= proj.new_dataitem(name="url_data_item",kind="table",path=URL)

In [ ]:
run = func.run(action="job",inputs={"url":di.key},outputs={"dataset":"dataset"})# local_execution=True

In [ ]:
run.status

In [ ]:
run.refresh()